In [ ]:
import os, sys
import glob
import altair as alt
alt.renderers.enable('notebook')
import pandas as pd
from pandas.io.json import json_normalize
import shared # analysis/shared

In [ ]:
print(os.getcwd())
data_6b94_128 = shared.load_data_files(glob.glob('../results/scale-cold-6b94-128-*.results'))
data_6b94_128.sort(key=lambda d: d[0]['size'])

In [ ]:
data_6b94_128[0][0]

In [ ]:
def process_data(data):
    processed = []
    for burst_data in data:
        datum = pd.DataFrame.from_dict(json_normalize(burst_data))
        for key in datum.keys():
            if 'timings' in key and 'phases' not in key:
                datum[key + "_zero"] = datum[key]
                datum[key + "_zero"] -= datum[key].min()
                datum[key + "_local"] = datum[key]
                datum[key + "_local"] -= datum["timings.start"].min()
        datum = datum.rename(index=str, columns={k: k.replace(".", "_") for k in datum.keys()})
        datum['timings_rtt'] = datum['timings_response'] - datum['timings_upload']
        processed.append(datum)
    return processed
pd_6b94_128 = process_data(data_6b94_128)
pd_6b94_128[0].keys()

In [ ]:
for burst_data in pd_6b94_128:
    print('Checking burst durations for b{}'.format(burst_data['size'][0]))
    display(alt.Chart(burst_data).mark_point(color='red').encode(
        alt.X('timings_start_local:Q', scale=alt.Scale(zero=False)),
        alt.Y('timings_rtt:Q'),
    ) + alt.Chart(burst_data).mark_point(color='orange').encode(
        alt.X('timings_lookup_local:Q'),
        alt.Y('timings_rtt:Q'),
    ) + alt.Chart(burst_data).mark_point(color='green').encode(
        alt.X('timings_connect_local:Q'),
        alt.Y('timings_rtt:Q'),
    ) + alt.Chart(burst_data).mark_point(color='blue').encode(
        alt.X('timings_upload_local:Q'),
        alt.Y('timings_rtt:Q'),
    ) + alt.Chart(burst_data).mark_point(color='purple').encode(
        alt.X('timings_response_local:Q'),
        alt.Y('timings_rtt:Q'),
    ).properties(
        title='b{}'.format(burst_data['size'][0])
    ).interactive())

In [ ]:
def chart_timings(data):
    for burst_data in data:
        print('Checking burst durations for b{}'.format(burst_data['size'][0]))
        display(alt.Chart(burst_data).mark_point(color='red').encode(
            alt.X('timings_start_local:Q', scale=alt.Scale(zero=False)),
            alt.Y('timings_rtt:Q'),
        ) + alt.Chart(burst_data).mark_point(color='orange').encode(
            alt.X('timings_lookup_local:Q'),
            alt.Y('timings_rtt:Q'),
        ) + alt.Chart(burst_data).mark_point(color='green').encode(
            alt.X('timings_connect_local:Q'),
            alt.Y('timings_rtt:Q'),
        ) + alt.Chart(burst_data).mark_point(color='blue').encode(
            alt.X('timings_upload_local:Q'),
            alt.Y('timings_rtt:Q'),
        ) + alt.Chart(burst_data).mark_point(color='purple').encode(
            alt.X('timings_response_local:Q'),
            alt.Y('timings_rtt:Q'),
        ).properties(
            title='b{}'.format(burst_data['size'][0])
        ).interactive())

def visualize_results(glob_path):
    data = shared.load_data_files(glob.glob(glob_path))
    data.sort(key=lambda d: d[0]['size'])
    pdata = process_data(data)
    chart_timings(pdata)
    
visualize_results('../http-test-original.results')

In [ ]:
visualize_results('../http-test-keep-alive.results')

In [ ]:
visualize_results('../http-test-keep-alive-warming.results')